In [ ]:
#default_exp actions

# GitHub Actions

> Functionality for helping to create GitHub Actions workflows in Python

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
from fastcore.meta import *
from ghapi.core import *

from enum import Enum

In [ ]:
#hide
from nbdev import *

## Workflow setup

In your GitHub Actions workflow, include the following in your `run` step:

```bash
env:
  CONTEXT_GITHUB: ${{ toJson(github) }}
```

This stores the full [github context](https://docs.github.com/en/free-pro-team@latest/actions/reference/context-and-expression-syntax-for-github-actions#github-context), which includes information such as the name of the current workflow being run, the GitHub access token, and so forth.

As well as the `github` context, you can do that same thing for any of the other GitHub Actions contexts, which are:

`github` `env` `job` `steps` `runner` `secrets` `strategy` `matrix` `needs`

For instance, for the [needs](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-syntax-for-github-actions#jobsjob_idneeds) context, information about previous jobs specified in your `needs` clause, add this underneath your `CONTEXT_GITHUB` line:

```bash
  CONTEXT_NEEDS: ${{ toJson(needs) }}
```

Note that here's no harm having entries that are not used -- GitHub Actions will set them to an empty dictionary by default.

In [ ]:
#export
# So we can run this outside of GitHub actions too, read from file if needed
if 'CONTEXT_GITHUB' not in os.environ:
    os.environ['CONTEXT_GITHUB'] = Path('examples/context.json').read_text()
if 'CONTEXT_GITHUB' not in os.environ:
    os.environ['CONTEXT_NEEDS'] = Path('examples/needs.json').read_text()

In [ ]:
#export
contexts = 'github', 'env', 'job', 'steps', 'runner', 'secrets', 'strategy', 'matrix', 'needs'
for context in contexts:
    globals()[f'context_{context}'] = dict2obj(loads(os.getenv(f"CONTEXT_{context.upper()}", "{}")))

In [ ]:
#export
_all_ = ['context_github', 'context_env', 'context_job', 'context_steps', 'context_runner', 'context_secrets', 'context_strategy', 'context_matrix', 'context_needs']

In [ ]:
# export
_wf_tmpl = """
name: NAME
on:
  workflow_dispatch:
  EVENT:
defaults:
  run: { shell: bash }

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v1
    - uses: actions/setup-python@v2
      with: {python-version: '3.8'}
    - name: Run script
      env:
        CONTEXT_GITHUB: ${{ toJson(github) }}
      run: |
        pip install -q ghapi
        python .github/scripts/build-NAME-EVENT.py
"""

In [ ]:
# export
def create_workflow(name:str, event:'Event'):
    "Create a simple Ubuntu workflow that calls a Python `ghapi` script"
    if not os.path.exists('.git'): return print('This does not appear to be the root of a git repo')
    wf_path  = Path('.github/workflows')
    scr_path = Path('.github/scripts')
    wf_path .mkdir(parents=True, exist_ok=True)
    scr_path.mkdir(parents=True, exist_ok=True)
    fname = f'{name}-{event}'
    if not (wf_path/f'{fname}.yml').exists():
        contents = _wf_tmpl.replace('NAME',name).replace('EVENT',str(event))
        (wf_path/f'{fname}.yml').write_text(contents)
    if not (scr_path/f'build-{fname}.py').exists():
        py = "from fastcore.all import *\nfrom ghapi import *"
        (scr_path/f'build-{fname}.py').write_text(py)

To create a basic skeleton of a `ghapi` workflow, call `create_workflow`, passing in the event that you wish to respond to, and a name for your workflow.

## Accessing contexts from Python

The information from these variables are provided by `context_github`, `context_needs`, and so forth for each named context. These variables are `AttrDict` objects.

In [ ]:
L(context_github)

(#26) ['token','job','ref','sha','repository','repository_owner','repositoryUrl','run_id','run_number','retention_days'...]

In [ ]:
context_github.ref

'refs/heads/master'

If you use our recommended workflow template, you will have this included in your prebuild step (if you have any):

```bash
outputs:
  out: ${{ toJson(steps) }}
```

You can access this content as a dictionary like so:

In [ ]:
loads(nested_idx(context_needs, "prebuild", "outputs", "out"))

{'step1': {'outputs': {'action_fruit': 'strawberry'},
  'outcome': 'success',
  'conclusion': 'success'}}

In [ ]:
#export
_example_url = 'https://raw.githubusercontent.com/fastai/ghapi/master/examples/{}.json'

In [ ]:
#export
def example_payload(event):
    "Get an example of a JSON payload for `event`"
    return dict2obj(urljson(_example_url.format(event)))

In [ ]:
#hide
print(','.join(repr(o) for o in Path('examples/').ls(file_exts=['.json']).attrgot('stem')))

'page_build','content_reference','repository_import','create','workflow_run','delete','organization','sponsorship','project_column','push','context','milestone','project_card','project','package','pull_request','repository_dispatch','team_add','workflow_dispatch','member','meta','code_scanning_alert','public','needs','check_run','security_advisory','pull_request_review_comment','org_block','commit_comment','watch','marketplace_purchase','star','installation_repositories','check_suite','github_app_authorization','team','status','repository_vulnerability_alert','pull_request_review','label','installation','release','issues','repository','gollum','membership','deployment','deploy_key','issue_comment','ping','deployment_status','fork'


The possible events are available in the `Event` `enum`.

In [ ]:
#export
Event = str_enum('Event',
    'page_build','content_reference','repository_import','create','workflow_run','delete','organization','sponsorship',
    'project_column','push','context','milestone','project_card','project','package','pull_request','repository_dispatch',
    'team_add','workflow_dispatch','member','meta','code_scanning_alert','public','needs','check_run','security_advisory',
    'pull_request_review_comment','org_block','commit_comment','watch','marketplace_purchase','star','installation_repositories',
    'check_suite','github_app_authorization','team','status','repository_vulnerability_alert','pull_request_review','label',
    'installation','release','issues','repository','gollum','membership','deployment','deploy_key','issue_comment','ping',
    'deployment_status','fork')

In [ ]:
', '.join(Event)

'page_build, content_reference, repository_import, create, workflow_run, delete, organization, sponsorship, project_column, push, context, milestone, project_card, project, package, pull_request, repository_dispatch, team_add, workflow_dispatch, member, meta, code_scanning_alert, public, needs, check_run, security_advisory, pull_request_review_comment, org_block, commit_comment, watch, marketplace_purchase, star, installation_repositories, check_suite, github_app_authorization, team, status, repository_vulnerability_alert, pull_request_review, label, installation, release, issues, repository, gollum, membership, deployment, deploy_key, issue_comment, ping, deployment_status, fork'

## Workflow helper functions

In [ ]:
#export
def github_token():
    "Get GitHub token from `GITHUB_TOKEN` env var if available, or from `github` context"
    return os.getenv('GITHUB_TOKEN', context_github.get('token', None))

In [ ]:
#export
def actions_output(name, value):
    "Print the special GitHub Actions `::set-output` line for `name::value`"
    print(f"::set-output name={name}::{value}")

Details in the [GitHub Documentation for `set-output`](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#setting-an-output-parameter).

In [ ]:
#export
def actions_debug(message):
    "Print the special `::debug` line for `message`"
    print(f"::debug::{message}")

Details in the [GitHub Documentation for `debug`](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#setting-a-debug-message). Note that you must create a secret named `ACTIONS_STEP_DEBUG` with the value true to see the debug messages set by this command in the log.

In [ ]:
#export
def actions_warn(message, details=''):
    "Print the special `::warning` line for `message`"
    print(f"::warning {details}::{message}")

Details in the [GitHub Documentation for `warning`](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#setting-a-warning-message). For the optional `details`, you can provide comma-delimited file, line, and column information, e.g.: `file=app.js,line=1,col=5`.

In [ ]:
#export
def actions_error(message, details=''):
    "Print the special `::error` line for `message`"
    print(f"::error {details}::{message}")

Details in the [GitHub Documentation for `error`](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#setting-an-error-message). For the optional `details`, you can provide comma-delimited file, line, and column information, e.g.: `file=app.js,line=1,col=5`.

In [ ]:
#export
def actions_group(title):
    "Print the special `::group` line for `title`"
    print(f"::group::{title}")

In [ ]:
#export
def actions_endgroup():
    "Print the special `::endgroup`"
    print(f"::endgroup::")

Details in the GitHub Documentation for [grouping log lines](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#grouping-log-lines).

In [ ]:
#export
def actions_mask(value):
    "Print the special `::add-mask` line for `value`"
    print(f"::add-mask::{value}")

Details in the [GitHub Documentation for `add-mask`](https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-commands-for-github-actions#masking-a-value-in-log).

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_actions.ipynb.
Converted 10_cli.ipynb.
Converted 50_fullapi.ipynb.
Converted 90_build_lib.ipynb.
Converted index.ipynb.
Converted tutorial_actions.ipynb.
Converted tutorial_api.ipynb.
